In [5]:
from torch.utils.data import Dataset, DataLoader
from datasets import load_dataset
from transformers import AutoTokenizer
import random
from prompt_templates import create_prompt
from transformers import MT5ForConditionalGeneration
import argparse
from datetime import datetime
from models.get_model import get_model
from dataloader import get_loader
import torch
from torch import nn
from evaluate import evaluate
import copy
import wandb
from tqdm import tqdm
import os
import json
from torch.nn import CrossEntropyLoss
from transformers import AutoTokenizer
from scheduler import CosineWarmupScheduler
from utils import compute_rank_loss
from torch.cuda.amp import GradScaler


In [6]:
%load_ext autoreload
%autoreload 2

In [4]:
tokenizer = AutoTokenizer.from_pretrained("google/mt5-small")
model = MT5ForConditionalGeneration.from_pretrained("google/mt5-small")

In [44]:
class b:
    dataset = "demo"
    batch_size = 3
    num_workers = 4

b = b()

In [18]:
data_loader = get_loader(b, 'train', tokenizer, T=4, debug=True)

In [45]:
data_loader2 = get_loader(b, 'train', tokenizer, T=4, debug=False)


In [46]:
out1= next(iter(data_loader2))

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
Asking to truncate to max_le

In [64]:
out1['pos_input_ids'].shape, out1['pos_attention_mask'].shape, out1['neg_input_ids'].shape, out1['pos_labels'].shape, out1['neg_attention_mask'].shape

(torch.Size([3, 230]),
 torch.Size([3, 230]),
 torch.Size([3, 223]),
 torch.Size([3, 2]),
 torch.Size([3, 223]))

In [51]:
for i in range(3):
    print(tokenizer.decode(out1['pos_input_ids'][i].tolist()))
    print(tokenizer.decode(out1['neg_input_ids'][i].tolist()))  

En bruger har for nylig læst artikler: Tomas Axnér bliver ny cheftræner i Team Esbjerg fra sommeren 2024, hvor han afløser Jesper Jensen. Svenskeren skal sideløbende være landstræner, Rafael Nadal har indkaldt til pressemøde torsdag. Ifølge AS og Marca vil han annoncere French Open-afbud, James Milner, Roberto Firmino, Naby Keita og Alex Oxlade-Chamberlain stopper i Liverpool efter denne sæson, Rafael Nadal har indkaldt til pressemøde torsdag. Ifølge AS og Marca vil han annoncere French Open-afbud, vil brugeren læse artiklen God fangst på havn trods muligt forsøg på at aflede? (ja/nej)</s><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad>
En bruger har for nylig læst artikler: Tomas Axnér bliver ny cheftræner i Team Esbjerg fra sommeren 2024, hvor han afl

In [65]:
pos_outputs = model(
    input_ids=out1["pos_input_ids"], 
    attention_mask=out1["pos_attention_mask"],
)
pos_outputs.encoder_last_hidden_state.shape

ValueError: You have to specify either decoder_input_ids or decoder_inputs_embeds

In [67]:
tokenizer.encode("ja / nej"), tokenizer.encode("ja"), tokenizer.encode("nej")

([432, 259, 275, 3810, 1], [432, 1], [3810, 1])

In [62]:
neg_outputs = model(
    input_ids=out1["neg_input_ids"], 
    attention_mask=out1["neg_attention_mask"],
    labels=out1["neg_labels"],
)
neg_outputs.encoder_last_hidden_state.shape

torch.Size([3, 223, 512])

In [54]:
pos_outputs.logits.shape

torch.Size([3, 2, 250112])

In [30]:
out1.keys()

dict_keys(['pos_input_ids', 'pos_attention_mask', 'neg_input_ids', 'neg_attention_mask', 'pos_labels', 'neg_labels', 'empt'])

In [43]:
print(tokenizer.decode(out1['pos_input_ids'][0].tolist()))
print(tokenizer.decode(out1['neg_input_ids'][0].tolist()))

En bruger har for nylig læst artikler: Tennis-etteren Novak Djokovic havde flotte ord om Holger Rune på pressemødet efter de to tennisstjerners dyst i Rom og fortæller, at det er ham, der skal have råd fra Rune og ikke omvendt, Rafael Nadal har indkaldt til pressemøde torsdag. Ifølge AS og Marca vil han annoncere French Open-afbud, Rygtemøllen arbejder på højtryk, efter Belarus' præsident, Alexander Lukasjenko, ikke er blevet set offentligt i en uge, Luksuslivet er slut. Bruger penge på Easyjet, sandkage og cola, vil brugeren læse artiklen - Hvis nogen tror, at vi lægger os fladt ned mod FC Nordsjælland, så må det være personer, der ikke er involveret i elitesport, siger en vred Carsten V. Jensen? (ja/nej)</s>
En bruger har for nylig læst artikler: Tennis-etteren Novak Djokovic havde flotte ord om Holger Rune på pressemødet efter de to tennisstjerners dyst i Rom og fortæller, at det er ham, der skal have råd fra Rune og ikke omvendt, Rafael Nadal har indkaldt til pressemøde torsdag. If

"En bruger har for nylig læst artikler: Tennis-etteren Novak Djokovic havde flotte ord om Holger Rune på pressemødet efter de to tennisstjerners dyst i Rom og fortæller, at det er ham, der skal have råd fra Rune og ikke omvendt, Rafael Nadal har indkaldt til pressemøde torsdag. Ifølge AS og Marca vil han annoncere French Open-afbud, Rygtemøllen arbejder på højtryk, efter Belarus' præsident, Alexander Lukasjenko, ikke er blevet set offentligt i en uge, Luksuslivet er slut. Bruger penge på Easyjet, sandkage og cola, vil brugeren læse artiklen Aktien er faldet med knap 30 procent onsdag? (ja/nej)</s>"

In [37]:
tokenizer.encode("hello")

[124393, 1]

In [27]:
out1, out2 = next(iter(data_loader2)), next(next(iter(data_loader2)))

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.
Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.
Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.
/home/wouter/miniconda3/envs/RecSys/lib/python3.11/site-packages/transformers/tokenization_utils_base.py:3946: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(
/home/wouter/miniconda3/envs/RecSys/lib/python3.11/site-packages/transformers/tokenization_utils_base.py:3946: UserWarning: `as_target_tokenizer` is dep

TypeError: 'dict' object is not an iterator